**Title**: Data Wrangling 11.2 Exercises  
**Author**: Ryan Weeks  
**Date**: 2/22/2025  
**Description**:  These exercises focused on using SQLAlchemy to create and manage a database for storing and querying sensor and station data. I defined tables, inserted records, and queried them using SQLAlchemy’s ORM capabilities. I also worked on creating relationships between the sensor and station tables, which helped me understand how data can be linked and queried efficiently. I used both SQL queries and pandas to analyze and retrieve data from the database. It was a great hands-on experience that gave me a deeper understanding of database management and querying while exploring the practical application of SQLAlchemy!

# Import sqlite3 and create a cursor

In [3]:
import sqlite3

# Using an in-memory database
with sqlite3.connect(":memory:") as con:
    c = con.cursor()

    # Create the 'sensors' table
    c.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')

    # Confirm the table was created by querying the table names
    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table:", table[0])

    # Insert random data into the 'sensors' table
    c.execute("INSERT INTO sensors VALUES ('2016-11-05', 'Utrecht', 'Red', 42, 15.14)")

    # Select all records to see the data inserted
    c.execute("SELECT * FROM sensors")
    print("Data in 'sensors' table:", c.fetchall())

    # Drop the table
    con.execute("DROP TABLE sensors")

    # Confirm that the table has been dropped
    print("# of tables after drop:", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    # No need to explicitly call c.close() as 'with' handles it

Table: sensors
Data in 'sensors' table: [('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)]
# of tables after drop: 0


# Create a list of tuples to form the pandas DataFrame

In [5]:
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

# Using an in-memory SQLite database
with sqlite3.connect(":memory:") as con:
    c = con.cursor()

    # Load the sunspots dataset from statsmodels
    data_loader = sm.datasets.sunspots.load_pandas()
    df = data_loader.data

    # Convert the dataframe to a list of tuples for insertion
    rows = [tuple(x) for x in df.values]

    # Create the sunspots table without specifying data types (it's fine for this example)
    con.execute("CREATE TABLE sunspots(year, sunactivity)")

    # Insert all rows into the table
    con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)

    # Show the row count after inserting
    c.execute("SELECT COUNT(*) FROM sunspots")
    print("Total rows in the table:", c.fetchone()[0])

    # Delete records where sunactivity is greater than 20 and show how many were deleted
    deleted_rows = con.execute("DELETE FROM sunspots WHERE sunactivity > 20").rowcount
    print(f"Deleted {deleted_rows} rows where sunactivity > 20")

    # Select and show all data where year is less than 1732
    print("Data with year < 1732:")
    print(read_sql("SELECT * FROM sunspots WHERE year < 1732", con))

    # Drop the table after use
    con.execute("DROP TABLE sunspots")

    # No need to manually close the cursor since 'with' handles it

Total rows in the table: 309
Deleted 217 rows where sunactivity > 20
Data with year < 1732:
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


# Install sqlalchemy and Create an engine from a URI

In [7]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker, declarative_base
import pandas as pd

# Define the base class for our models
Base = declarative_base()

# Define the Station class (superclass)
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

    # Define a relationship between Station and Sensor (One-to-Many)
    sensors = relationship("Sensor", back_populates="station")

# Define the Sensor class
class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True)
    type = Column(String, nullable=False)
    station_id = Column(Integer, ForeignKey('stations.id'))

    # Define a relationship to back-reference to Station
    station = relationship("Station", back_populates="sensors")

# Create the engine (using SQLite in this example)
engine = create_engine('sqlite:///stations.db')

# Create the tables in the database
Base.metadata.create_all(engine)

# Create a session maker bound to the engine
Session = sessionmaker(bind=engine)
session = Session()

# Create two station records
station1 = Station(name="Station A")
station2 = Station(name="Station B")

# Add and commit stations to the session
session.add(station1)
session.add(station2)
session.commit()

# Insert a sensor record related to station1
sensor1 = Sensor(type="Temperature", station_id=station1.id)
session.add(sensor1)
session.commit()

# Query: Select the first row in the station table
first_station = session.query(Station).first()
print("First Station:", first_station.name)

# Query: Select all stations
all_stations = session.query(Station).all()
print("All Stations:")
for station in all_stations:
    print(station.name)

# Query: Select all sensors
all_sensors = session.query(Sensor).all()
print("All Sensors:")
for sensor in all_sensors:
    print(f"Sensor {sensor.id}: {sensor.type}, Station {sensor.station.name}")

# Query: Select the first sensor that belongs to the first station
first_sensor = session.query(Sensor).filter(Sensor.station_id == first_station.id).first()
print(f"First Sensor for Station {first_station.name}: {first_sensor.type}")

# Query: Select all stations using pandas read_sql()
stations_df = pd.read_sql("SELECT * FROM stations", con=engine.connect())
print("Stations DataFrame:")
print(stations_df)

# Close the session
session.close()

First Station: Station A
All Stations:
Station A
Station B
Station A
Station B
Station A
Station B
All Sensors:
Sensor 1: Temperature, Station Station A
Sensor 2: Temperature, Station Station A
Sensor 3: Temperature, Station Station A
First Sensor for Station Station A: Temperature
Stations DataFrame:
   id       name
0   1  Station A
1   2  Station B
2   3  Station A
3   4  Station B
4   5  Station A
5   6  Station B
